# Deploy RoBERTa Embedding (CPU) Model Package from AWS Marketplace 


This sample notebook shows you how to deploy [RoBERTa Embedding (CPU)]( https://aws.amazon.com/marketplace/pp/prodview-nvrw726ubwj26) using Amazon SageMaker. You could use the embedding for semantic similarity and building RAG solutions.

This listing is specifically to run on CPU instances. If you would like to deploy this package on a GPU instance, use the following [RoBERTa Embedding model package](https://aws.amazon.com/marketplace/pp/prodview-pwfcefr5utpwg)


## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
   1. **aws-marketplace:ViewSubscriptions**
   1. **aws-marketplace:Unsubscribe**
   1. **aws-marketplace:Subscribe**  

## Contents:


You can deploy the package via the UI or programmatically. After the deployment is complete you should have an API endpoint that you can query with your data in your private cloud. Below are the steps to use the model package via both the methods. We suggest starting with the UI method since that's the simplest.


### Deploy via UI

1. [Subscribe to the model package from the UI](#1.-Subscribe-to-the-model-package-from-the-UI)
2. [Invoke the endpoint via CLI](#2.-Invoke-the-endpoint-via-CLI)

### Deploy Programmatically (Optional)
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

# Deploy via UI


## 1. Subscribe to the model package from the UI

To subscribe to the model package:
1. Open the model package listing page:[RoBERTa Embedding](https://aws.amazon.com/marketplace/pp/prodview-nvrw726ubwj26).
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button**, choose one of the recommended launch methods. We recommend using AWS Cloudformation launch method for seamless deployment without any manual steps. Choose one of the existing sagemaker IAM roles or let the UI create a new role. Click on the **Launch CloudFormation Template** to begin creation of Sagemaker endpoint. 
1. Next on the **Create Stack** page, review the parameters and click on the **Create stack** button. This should start the creation of stack which will create a sagemaker endpoint ready to be queried. 

![title](./imgs/subscribe.png)

![title](./imgs/launch-method-roberta-cpu.png)

Note the name of the sagemaker endpoint created. It should be visible under the **AWS Sagemaker Console > Inference > Endpoints** as shown in the image below

![title](./imgs/endpoint-name.png)

Save the name of the endpoint in a variable

In [1]:
endpoint_name = "Endpoint-RoBERTa-Embedding-1"

## 2. Invoke the endpoint via CLI

The endpoint takes input in the following format `'{"sentences": ["Hello, how are you doing?", "I am doing great, how about you?"]}'`. It takes an array of sentences and returns an array of embedding corresponding to each of the sentences. 

In [2]:
# Escape the text(body) to be able to pass it from the CLI
body = """'{"sentences": ["Hello, how are you doing?", "I am doing great, how about you?"]}'"""
content_type = "application/json"
region = "us-east-1"

The output from the endpoint is written to the file output.json

In [3]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $endpoint_name \
    --body $body \
    --content-type $content_type \
    --region $region \
    output.json

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}


Your sagemaker endpoint is ready to be queried and used! 

In [4]:
# Check the output from the endpoint
!head -c 100 output.json

{"embeddings":[[-0.066679447889328,0.19491353631019592,0.03849916160106659,-0.0400514155626297,0.263

# Deploy programmatically


## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page:[RoBERTa Embedding](https://aws.amazon.com/marketplace/pp/prodview-nvrw726ubwj26).
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [5]:
model_package_arn = "arn:aws:sagemaker:us-east-1:865070037744:model-package/robertembeddingmodel-b89cf0d12461354199f7436958cd2800"

In [6]:
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[01/25/25 02:21:07] INFO     Found credentials from IAM Role:                                   ]8;id=683096;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=951400;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [7]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")
bucket

                    INFO     Found credentials from IAM Role:                                   ]8;id=378428;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=135896;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=561598;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=92969;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[01/25/25 02:21:08] INFO     Found credentials from IAM Role:                                   ]8;id=172173;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=142071;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

'sagemaker-us-east-1-809103429415'

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [8]:
model_name = "roberta-embedding"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.m4.xlarge"
)
batch_transform_inference_instance_type = (
    "ml.m4.xlarge"
)

### A. Create an endpoint

In [9]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

[01/25/25 02:21:38] INFO     Creating model with name:                                              ]8;id=643348;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=965333;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\
                             robertembeddingmodel-b89cf0d12461354199-2025-01-25-02-21-38-673                       

[01/25/25 02:21:43] INFO     Creating endpoint-config with name roberta-embedding                   ]8;id=654343;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=548378;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\

[01/25/25 02:21:44] INFO     Creating endpoint with name roberta-embedding                          ]8;id=413234;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=308158;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\

------------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Create input payload

In [10]:
expected_input_format = '{"sentences": ["Hello, how are you doing?", "I am doing great, how about you?"]}'
file_name = "input.json"
with open(file_name, "w") as f:
    f.write(expected_input_format)

<Add code snippet that shows the payload contents>

In [11]:
output_file_name = "output.json"

### C. Perform real-time inference

In [12]:
! aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$file_name --content-type $content_type --region $sagemaker_session.boto_region_name $output_file_name

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}


### D. Visualize output

In [13]:
!head -c 100 output.json

{"embeddings":[[-0.0666792094707489,0.19491320848464966,0.038498807698488235,-0.04005127772688866,0.

### E. Delete the endpoint

You can terminate the endpoint anytime as follows:

In [14]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

[01/25/25 02:29:59] INFO     Deleting endpoint with name: roberta-embedding                         ]8;id=96090;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=878144;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4855\4855]8;;\

                    INFO     Deleting endpoint configuration with name: roberta-embedding           ]8;id=216319;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=136765;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4865\4865]8;;\

## 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [15]:
ls -la  data/input/batch/

total 12
drwxrwxr-x 2 ec2-user ec2-user 4096 Jan 25 01:25 ./
drwxrwxr-x 4 ec2-user ec2-user 4096 Jan  6 22:17 ../
-rw-rw-r-- 1 ec2-user ec2-user  193 Jan 25 01:18 input.jsonl


In [16]:
# upload the batch-transform job input files to S3
transform_input_folder = "data/input/batch"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name)
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-1-809103429415/roberta-embedding


In [ ]:
# Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type, strategy='SingleRecord')
transformer.transform(transform_input, content_type=content_type, split_type='Line')
transformer.wait()

In [ ]:
# output is available on following path
transformer.output_path

The output path returned above should contain a file "input.jsonl.out" (input file name suffixed with .out). The output in this file must match the output available in data/output/batch folder.

## 4. Clean-up

### A. Delete the model

In [ ]:
model.delete_model()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

